In [60]:
from sympy import *
from sympy.abc import *

t=symbols('t')

def polinomioAutoCorrelazione(stringa): #metodo che ritorna il polinomio di autocorrelazione di una stringa in input
    #il primo termine del polinomio è sempre uno e per questo parto da una funzione che contiene solo uno
    def f(t1):
        return 1
    func=f(t)
    for i in range(len(stringa)-1): #ad ogni iterazione i confronto i primi n-i-1 caratteri della stringa (con n lunghezza della stringa) con gli ultimi n-i-1, se sono uguali aggiungo il termine t^i+1 al polinomio di autocorrelazione
        if stringa[i+1:]==stringa[:-(i+1)]:
            func= func + t ** (i+1) #se corrispondono aggiungo al polinomio il termine i+1-esimo
    return func

def avoidpattern(pattern, m): #metodo che restituisce la funzione generatrice delle parole dell'alfabeto di cardinalità m che non contengono il pattern passato in input
    func=polinomioAutoCorrelazione(pattern) #trovo il polinomio di autocorrelazione del pattern
    k=len(pattern)
    avoidfunc=func/(t**k+(1-m*t)*func)
    return avoidfunc

def containpattern(pattern, m): #metodo che restituisce la funzione generatrice delle parole dell'alfabeto di cardinalità m che contengono il pattern passato in input
    func=polinomioAutoCorrelazione(pattern) #trovo il polinomio di autocorrelazione del pattern
    k=len(pattern)
    containfunc=t**k/((1-m*t)*(t**k+(1-m*t)*func))
    return containfunc

In [61]:
polinomioAutoCorrelazione("aabbaa") #trovo il polinomio di autocorrelazione della stringa aabbaa

t**5 + t**4 + 1

In [120]:
f=avoidpattern("boh",26) #trovo la funzione generatrice delle parole dell'alfabeto binario {0,1} che non contengono il pattern 0010

In [121]:
f

1/(t**3 - 26*t + 1)

In [123]:
series(f,t,n=11) #sviluppo in serie la funzione generatrice trovata in precedenza per verificarne la correttezza, in particolare osserviamo come il coefficiente associato al termine di grado i corrisponda al numero di parole di lunghezza i che non contengono il pattern

1 + 26*t + 676*t**2 + 17575*t**3 + 456924*t**4 + 11879348*t**5 + 308845473*t**6 + 8029525374*t**7 + 208755780376*t**8 + 5427341444303*t**9 + 141102848026504*t**10 + O(t**11)

In [113]:
f1=containpattern("boh",26) #trovo la funzione generatrice delle parole dell'alfabeto binario {0,1} che contengono il pattern 0010

In [114]:
f1

t**3/((1 - 26*t)*(t**3 - 26*t + 1))

In [115]:
series(f1,t,n=11) # come prima sviluppo in serie la funzione generatrice trovata in precedenza per verificarne la correttezza, in particolare osserviamo che sommando i coefficienti associati ai termini t elevati alla stessa potenza n dei due sviluppi otterremo che il risultato sarà pari a 2^n, ossia il numero di possibili disposizioni con ripetizioni di {0,1} di lunghezza n, ossia tutte le parole di lunghezza n che è possibile generare con un alfabeto di cardinalità 2

t**3 + 52*t**4 + 2028*t**5 + 70303*t**6 + 2284802*t**7 + 71284200*t**8 + 2162234673*t**9 + 64247626872*t**10 + O(t**11)

In [118]:
import itertools
import random

def contaStringheConPattern(pattern, alfabeto, lungpar):
    paroletot=[''.join(i) for i in itertools.product(alfabeto, repeat = lungpar)]
    count = sum([1 for parola in paroletot if pattern in parola])
    return count

def contaStringheSenzaPattern(pattern, alfabeto, lungpar):
    paroletot=[''.join(i) for i in itertools.product(alfabeto, repeat = lungpar)]
    count = sum([1 for parola in paroletot if pattern not in parola])
    return count

def generaParolaCasuale(alfabeto,lunghezza):
    return''.join(random.choices(alfabeto, k=lunghezza))

In [119]:
contaStringheSenzaPattern('boh',string.ascii_lowercase,5)

11879348

In [70]:
generaParolaCasuale(string.ascii_lowercase,5)

'dfrzh'

In [1]:
def approssimaRapportoParoleConPattern(pattern, alfabeto, lungpar, numit):
    count=0
    for i in range(1, numit):
        par=generaParolaCasuale(alfabeto,lungpar)
        if pattern in par:
            count=count+1
    return count/numit

def approssimaRapportoParoleSenzaPattern(pattern, alfabeto, lungpar, numit):
    count=0
    for i in range(1, numit):
        par=generaParolaCasuale(alfabeto,lungpar)
        if pattern not in par:
            count=count+1
    return count/numit

In [131]:
parcon=approssimaRapportoParoleConPattern('boh', string.ascii_lowercase, 10, 100000)
parcon

0.00043

In [132]:
parsenz=approssimaRapportoParoleSenzaPattern('boh', string.ascii_lowercase, 10, 100000)
parsenz

0.9996

In [133]:
parcon+parsenz

1.00003